In [1]:
import random, pickle, json, logging
from abc import ABC, abstractmethod

logging.basicConfig(filename="log.txt", level=logging.INFO)

# ==== State Pattern ====
class State(ABC):
    @abstractmethod
    def action(self, player):
        pass

class HealthyState(State):
    def action(self, player):
        logging.info("Player is healthy.")

class PoisonedState(State):
    def action(self, player):
        player.health -= 5
        logging.info("Player is poisoned! Health -5.")

class DeadState(State):
    def action(self, player):
        logging.info("Player is dead. No actions possible.")

# ==== Base Object ====
class GameObject:
    def __init__(self, name):
        self.name = name

# ==== Items ====
class Item(GameObject):
    def __init__(self, name, effect):
        super().__init__(name)
        self.effect = effect

    def use(self, player):
        if self.effect == "heal":
            player.health += 20
            logging.info(f"{player.name} used {self.name} (+20 health)")

# ==== Quest ====
class Quest:
    def __init__(self, quest_type):
        self.type = quest_type
        self.completed = False

    def check(self, player):
        # Simplified
        if self.type == "kill monster":
            self.completed = True
        logging.info(f"Quest checked: {self.type} - {self.completed}")
        return self.completed

# ==== Player & Enemy ====
class Character(GameObject):
    def __init__(self, name, strength, defense, health):
        super().__init__(name)
        self.strength = strength
        self.defense = defense
        self.health = health
        self.state = HealthyState()

class Player(Character):
    def __init__(self, name):
        super().__init__(name, 10, 5, 100)
        self.inventory = []
        self.state = HealthyState()

    def attack(self, enemy):
        damage = max(self.strength - enemy.defense, 1)
        enemy.health -= damage
        logging.info(f"{self.name} attacks {enemy.name} for {damage} damage")

    def defend(self):
        self.defense += 2
        logging.info(f"{self.name} defends (defense +2)")

    def use_item(self, item):
        item.use(self)
        self.inventory.remove(item)

    def update_state(self):
        if self.health <= 0:
            self.state = DeadState()
        elif self.health < 30:
            self.state = PoisonedState()
        else:
            self.state = HealthyState()
        self.state.action(self)

class Enemy(Character):
    def decide_action(self, player):
        if self.health < 20:
            logging.info(f"{self.name} retreats!")
        else:
            self.attack(player)

    def attack(self, player):
        damage = max(self.strength - player.defense, 1)
        player.health -= damage
        logging.info(f"{self.name} attacks {player.name} for {damage} damage")

# ==== World ====
class World:
    def __init__(self):
        self.zones = ["forest", "dungeon", "castle"]
        self.quests = [Quest(random.choice(["kill monster", "top item", "explore zone"])) for _ in range(3)]

# ==== Save / Load ====
def save_game(player, filename="savegame.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(player, f)
    logging.info("Game saved.")

def load_game(filename="savegame.pkl"):
    with open(filename, "rb") as f:
        player = pickle.load(f)
    logging.info("Game loaded.")
    return player

# ==== Main ====
def main():
    player = Player("Hero")
    enemy = Enemy("Goblin", 7, 3, 30)
    potion = Item("Health Potion", "heal")
    player.inventory.append(potion)

    player.attack(enemy)
    enemy.decide_action(player)
    player.use_item(potion)
    player.update_state()

    save_game(player)
    loaded_player = load_game()

if __name__ == "__main__":
    main()

